## Cache and paralelization example

This notebook runs a set of examples on how to use the OptionValuationContext class. It also shows how to use the cache and paralelization to speed up computations. It is intentionally verbose with prints and small plots to help debugging.

### Notes:
 - Run from the repository notebooks folder. The first cell will take you to the root folder of the project so the necessary modules can be imported

In [1]:
import os

# --- Step 1: Define the target folder relative to the current folder ---
# '..' means go up one level. '..\\..' means go up two levels.
relative_path_up_two = os.path.join('..', '..') 

# --- Step 2: Navigate up two levels (this assumes the notebook's current 
#             folder is the working directory, which is typical in Jupyter)
try:
    # Change the working directory to the target path
    os.chdir(relative_path_up_two)
    
    # Print the new directory for confirmation
    print(f"Successfully changed directory up two levels to: \n{os.getcwd()}")
    
except FileNotFoundError:
    print("Error: Could not navigate up two levels. Check your starting path.")

Successfully changed directory up two levels to: 
c:\Users\ramon\OneDrive\Desktop\Python projects\OptionPricingPY


In [ ]:
from src.engines import *
from src.products import *
from src.models import *
from src.greeks import *
from src.valuation import *

import numpy as np
import logging
import pandas as pd

# Setup logging para ver los hooks
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

print("="*70)
print("DEMO: OptionValuationContext con logging, cache y paralelización")
print("="*70)

# ============================================================================
# PARTE 1: Single valuation con Context vs Raw Engine
# ============================================================================
print("\n" + "="*70)
print("PARTE 1: Single valuation - Context vs Raw Engine")
print("="*70)

expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=30)
option = EuropeanOption(S=100, K=100, expiry_date=expiry_date, option_type='call', qty=1)
bs_model = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)

# Raw engine (sin context)
print("\n[RAW ENGINE] Analytical:")
analytical_engine = AnalyticalEngine()
price_raw = analytical_engine.calculate_price(option, bs_model)
print(f"  Precio: {price_raw:.6f}")

# Con context (sin cache)
print("\n[CONTEXT] Analytical (sin cache):")
ctx_analytical = OptionValuationContext(analytical_engine, cache_enabled=False)
price_ctx = ctx_analytical.value_option(option, bs_model)
print(f"  Precio: {price_ctx:.6f}")

# Verificar que son iguales
assert abs(price_raw - price_ctx) < 1e-10, f"Mismatch! {price_raw} vs {price_ctx}"
print(f"  ✓ Precios coinciden: {price_raw:.6f} == {price_ctx:.6f}")

# ============================================================================
# PARTE 2: Cache behavior
# ============================================================================
print("\n" + "="*70)
print("PARTE 2: Cache behavior (miss/hit)")
print("="*70)

ctx_cached = OptionValuationContext(analytical_engine, cache_enabled=True, cache_maxsize=128)

print("\n[CACHE] Primera llamada (MISS):")
p1 = ctx_cached.value_option(option, bs_model)
print(f"  Precio: {p1:.6f}")

print("\n[CACHE] Segunda llamada con mismo producto/modelo (HIT):")
p2 = ctx_cached.value_option(option, bs_model)
print(f"  Precio: {p2:.6f}")

assert p1 == p2, "Cache hit devolvió valor diferente"
print(f"  ✓ Cache funciona: {p1:.6f} == {p2:.6f}")

print("\n[CACHE] Tercera llamada con K diferente (MISS):")
option2 = EuropeanOption(S=100, K=110, T=30, option_type='call', qty=1)
p3 = ctx_cached.value_option(option2, bs_model)
print(f"  Precio: {p3:.6f}")
assert p3 != p1, "Cache no diferencia strikes"
print(f"  ✓ Cache diferencia strikes: {p1:.6f} != {p3:.6f}")

# ============================================================================
# PARTE 3: Batch valuation sin paralelización
# ============================================================================
print("\n" + "="*70)
print("PARTE 3: Batch valuation (secuencial)")
print("="*70)

strikes = np.linspace(80, 120, 10)
products_batch = [EuropeanOption(S=100, K=K, T=30, option_type='call', qty=1) for K in strikes]

print(f"\nValuando {len(products_batch)} opciones (secuencial)...")
ctx_batch = OptionValuationContext(analytical_engine, cache_enabled=False, parallel=False)
prices_ctx_batch = ctx_batch.value_options(products_batch, bs_model)

print("\n[RAW ENGINE] Precios con loop:")
prices_raw_batch = [analytical_engine.calculate_price(p, bs_model) for p in products_batch]

print("\nComparación strikes:")
print(f"{'Strike':<10} {'Context':<15} {'Raw':<15} {'Diff':<15}")
print("-" * 55)
for K, p_ctx, p_raw in zip(strikes, prices_ctx_batch, prices_raw_batch):
    diff = abs(p_ctx - p_raw)
    print(f"{K:<10.2f} {p_ctx:<15.6f} {p_raw:<15.6f} {diff:<15.6e}")
    assert diff < 1e-10, f"Mismatch en K={K}"
print("✓ Todos los precios coinciden")

# ============================================================================
# PARTE 4: Batch valuation CON paralelización
# ============================================================================
print("\n" + "="*70)
print("PARTE 4: Batch valuation (paralelizado)")
print("="*70)

mc_engine = MonteCarloEngine(n_paths=50000, seed=42)

print(f"\nValuando {len(products_batch)} opciones (PARALELO, max_workers=4)...")

# Progress callback para ver ejecución
def progress_cb(idx, price):
    if price is not None:
        print(f"  [Progress] Índice {idx} completado: precio={price:.6f}")

ctx_parallel = OptionValuationContext(
    mc_engine,
    cache_enabled=False,
    parallel=True,
    max_workers=4
)

prices_parallel = ctx_parallel.value_options(
    products_batch,
    bs_model,
    progress_callback=progress_cb
)

print(f"\n✓ {len(prices_parallel)} precios computados")
print(f"  Rango: [{min(prices_parallel):.6f}, {max(prices_parallel):.6f}]")

# ============================================================================
# PARTE 5: Engine switching (ventaja principal del Context)
# ============================================================================
print("\n" + "="*70)
print("PARTE 5: Engine switching sin cambiar cliente")
print("="*70)

option_test = EuropeanOption(S=100, K=100, T=365, option_type='call', qty=1)
bs_test = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)

ctx = OptionValuationContext(AnalyticalEngine())

print("\n[ENGINE 1] Analytical:")
p_analytical = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_analytical:.6f}")

print("\n[ENGINE 2] FFT (cambio en tiempo de ejecución):")
ctx.engine = FFTEngine()
p_fft = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_fft:.6f}")
print(f"  Diferencia: {abs(p_analytical - p_fft):.6e}")

print("\n[ENGINE 3] Monte Carlo (cambio en tiempo de ejecución):")
ctx.engine = MonteCarloEngine(n_paths=100000, seed=42)
p_mc = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_mc:.6f}")
print(f"  Diferencia vs Analytical: {abs(p_analytical - p_mc):.6e}")

print("\n✓ Engine switching funciona correctamente sin cambiar código cliente")

# ============================================================================
# PARTE 6: Comparación Heston con múltiples engines
# ============================================================================
print("\n" + "="*70)
print("PARTE 6: Heston - FFT vs Monte Carlo vía Context")
print("="*70)

heston_model = HestonModel(
    kappa=2.0, theta=0.04, sigma=0.3, rho=-0.7, v0=0.04,
    r=0.05, q=0.02
)

option_heston = EuropeanOption(S=100, K=100, T=30, option_type='call', qty=1)

print("\n[CONTEXT + FFT] Heston:")
ctx_fft = OptionValuationContext(FFTEngine())
p_heston_fft = ctx_fft.value_option(option_heston, heston_model)
print(f"  Precio: {p_heston_fft:.6f}")

print("\n[CONTEXT + MC] Heston:")
ctx_mc = OptionValuationContext(MonteCarloEngine(n_paths=100000, seed=42))
p_heston_mc = ctx_mc.value_option(option_heston, heston_model)
print(f"  Precio: {p_heston_mc:.6f}")

diff_heston = abs(p_heston_fft - p_heston_mc)
print(f"\n  Diferencia: {diff_heston:.6f}")
print(f"  Diferencia relativa: {100 * diff_heston / p_heston_fft:.2f}%")

print("\n" + "="*70)
print("✓ DEMO COMPLETADA")
print("="*70)
#</VSCode.Cell>

DEBUG - Valuing product={'K': 100, 'T': 30, 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 2.40562374407466
DEBUG - Valuing product={'K': 100, 'T': 30, 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 2.40562374407466
DEBUG - Cache hit for key=130f2c1e77e514d67ef2df2daefd6e4b6d1c17d4f4c1a4536e7a4de0d6a1cf42
DEBUG - Valuing product={'K': 110, 'T': 30, 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 0.13312693878012816
DEBUG - Valuing product={'K': np.float64(80.0), 'T': 30, 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 20.163892661843278
DEBUG - Valuing product={'K': np.float64(84.44444444444444), 'T': 30, 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 15.73969319875944
DEBUG - V

DEMO: OptionValuationContext con logging, cache y paralelización

PARTE 1: Single valuation - Context vs Raw Engine

[RAW ENGINE] Analytical:
  Precio: 2.405624

[CONTEXT] Analytical (sin cache):
  Precio: 2.405624
  ✓ Precios coinciden: 2.405624 == 2.405624

PARTE 2: Cache behavior (miss/hit)

[CACHE] Primera llamada (MISS):
  Precio: 2.405624

[CACHE] Segunda llamada con mismo producto/modelo (HIT):
  Precio: 2.405624
  ✓ Cache funciona: 2.405624 == 2.405624

[CACHE] Tercera llamada con K diferente (MISS):
  Precio: 0.133127
  ✓ Cache diferencia strikes: 2.405624 != 0.133127

PARTE 3: Batch valuation (secuencial)

Valuando 10 opciones (secuencial)...

[RAW ENGINE] Precios con loop:

Comparación strikes:
Strike     Context         Raw             Diff           
-------------------------------------------------------
80.00      20.163893       20.163893       0.000000e+00   
84.44      15.739693       15.739693       0.000000e+00   
88.89      11.346537       11.346537       0.000000e

DEBUG - Price computed: 2.3937740115633916


  Precio: 2.393774

  Diferencia: 0.000154
  Diferencia relativa: 0.01%

✓ DEMO COMPLETADA
